In [1]:
using Flows
using ToySystems
using ToySystems.NineModeSystemEq
using Optim
using PyPlot; pygui(true)
include("9msctrl.jl")
include("Initial condition variation.jl")
include("Perturbation time.jl")
include("Contour Plots.jl")
include("Magnitude Rand.jl")
import LinearAlgebra: norm

┌ Info: Recompiling stale cache file C:\Users\User\.julia\compiled\v1.2\ToySystems\chsON.ji for ToySystems [53c88820-6eff-5a8c-9a74-badd815e7be1]
└ @ Base loading.jl:1240
┌ Warning: Package ToySystems does not have Flows in its dependencies:
│ - If you have ToySystems checked out for development and have
│   added Flows as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with ToySystems
└ Loading Flows into ToySystems from project dependency, future warnings for ToySystems are suppressed.


In [2]:
include("Objective function Monitor.jl")

objfun (generic function with 3 methods)

In [3]:
timestepdifference = 0.5
T = 10000


function AvObjInifrd(n, scale, Re, d::Float64)
    f  = NineModeSystem(Re)
    ϕ = flow(couple(f, objfun), RK4(couple(zeros(9), zeros(1))), TimeStepConstant(timestepdifference))
    output = zeros(n)
    for i in 1:n
        u0 = rand(9)
        u0_local = mgscale(u0, scale) + [d,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
        I0 = Float64[0.0]
        
        mon = Monitor(couple(zeros(9), zeros(1)), objfun)
        ϕ(couple(u0_local, I0), (0, T), reset!(mon))
        output[i] = I0[1]/T
    end
    PyPlot.plot(1:n,output)
    xlabel(L"n")
    ylabel(L"ObjectiveAverage")
end

function PInifrd(n, scale, Re, d::Float64, T, timestepdifference)
    output = zeros(n)
    for i in 1:n
        u0 = rand(9)
        u0_local = mgscale(u0, scale) + [d,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
        output[i] = P(u0_local, Re, T, timestepdifference)
    end
    PyPlot.plot(1:n,output)
    xlabel(L"n")
    ylabel(L"P(t)")   
end

PInifrd (generic function with 1 method)

In [4]:
AvObjInifrd(100, 0.1, 1000, 1.0) # 400 is the transition Re

PyObject Text(42.722222222222214, 0.5, '$ObjectiveAverage$')

In [5]:
PInifrd(1000,0.035,300,1.0, 10000, 0.5)

PyObject Text(20.847222222222214, 0.5, '$P(t)$')

In [4]:
function SurvivalProb(Re::AbstractVector, scale, n, t::AbstractVector, d::Float64, T, timestepdifference)
    u0_mat = zeros(9,n)
    for i in 1:n
        u0 = rand(9)
        u0_mat[:,i] = mgscale(u0, scale) + [d,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
    end #All condition
    
    PROBA = []
    for i in 1:length(Re) 
        Reynolds = Re[i] #fixed Re
        result = zeros(n) #time for all initial condition
        
        for j in 1:n
            u0_local = u0_mat[:,j]
            result[j] = P(u0_local, Reynolds, T, timestepdifference) #time for each initial condition
        end
        
        proba = zeros(length(t))
        for i in 1:length(t)
            condition = t[i]
            dummy = 0
            for j in 1:n
                if result[j] >= condition
                    dummy += 1
                else
                    dummy = dummy
                end
            end
            dummy = dummy/n
            proba[i] = dummy
        end
        plot(t, proba)
        PROBA = push!(PROBA, proba)
    end
    xlabel(L"t")
    ylabel(L"Prob(t)")
    return PROBA
end

SurvivalProb (generic function with 1 method)

In [5]:
y_val = SurvivalProb(200:50:450, sqrt(0.5), 1000, 0:100:10000, 0.0, 10000, 0.5) #Obj less than 0.01

6-element Array{Any,1}:
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]                    
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  0.002, 0.002, 0.001, 0.001, 0.001, 0.001, 0.0, 0.0, 0.0, 0.0]        
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  0.066, 0.064, 0.063, 0.059, 0.057, 0.056, 0.055, 0.053, 0.051, 0.05] 
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  0.262, 0.256, 0.249, 0.242, 0.237, 0.234, 0.231, 0.224, 0.217, 0.214]
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  0.531, 0.523, 0.517, 0.511, 0.508, 0.505, 0.503, 0.501, 0.496, 0.492]
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  0.706, 0.701, 0.695, 0.687, 0.68, 0.678, 0.671, 0.668, 0.666, 0.664] 

In [6]:
using PyCall
plt = pyimport("matplotlib.pyplot")

PyObject <module 'matplotlib.pyplot' from 'C:\\Users\\User\\.julia\\conda\\3\\lib\\site-packages\\matplotlib\\pyplot.py'>

In [7]:
fig, ax=  plt.subplots()
val = 50
for i in 1:length(y_val)
    ax.plot(0:100:10000, y_val[i], label = "Re = $val")
    val += 50
end
ax.legend()
xlabel(L"t")
ylabel(L"P(t)")

PyObject Text(42.722222222222214, 0.5, '$P(t)$')